In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import random
from os.path import exists
import torch
import torch.nn as nn

import torch.optim as optim

from datetime import datetime, timedelta
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
def DataClean(f1,f2):
	with open(f1) as f:
		data = json.load(f)
	for i in data.values():
		for j in i:
			del j["place_vector"]
			del j["city_vector"]
	json.dump(data,open(f2,'w'),ensure_ascii=False)

In [3]:
for i in range(3):
	if not exists('data_'+str(i)+'.json'):
		DataClean('classified_data_'+str(i)+'.json','data_'+str(i)+'.json')

In [4]:
# with open('data_0.json') as f:
# 	d0 = json.load(f)
# with open('data_1.json') as f:
# 	d1 = json.load(f)
# with open('data_2.json') as f:
# 	d2 = json.load(f)

In [5]:
def get_data(opt: int):
	with open('data_'+str(opt)+'.json') as f:
		data = json.load(f)
	return data
data = get_data(1)

In [6]:
input_size = 0
for city in data.values():
	if input_size != 0:
		break
	for p in city:
		if type(p['patient_vector']) == list:
			input_size = len(p['patient_vector'])
			break

In [7]:
input_size

300

In [8]:
total = 0
useful = 0
city_set = set()
for city in data.values():
	total += len(city)
	if len(city)>30:
		print(city[0]['true_city'])
		city_set.add(city[0]['true_city'])
	for p in city:
		if type(p['patient_vector']) == list:
			useful += 1


张掖
天水
兰州
重庆
德宏傣族景颇族自治州瑞丽
昆明
泉州
厦门
莆田
南京
苏州


In [9]:
# tset = set()
# for p in data['重庆']:
# 	tset.add(p['time'])
# tset

{'2021:07:20',
 '2021:07:24',
 '2021:07:27',
 '2021:07:28',
 '2021:07:29',
 '2021:10:26',
 '2021:10:27',
 '2021:10:28',
 '2021:10:29',
 '2021:10:30',
 '2021:10:31',
 '2021:11:01',
 '2021:11:02',
 '2021:11:03',
 '2022:02:26',
 '2022:02:27',
 '2022:02:28',
 '2022:03:02',
 '2022:03:04',
 '2022:03:05',
 '2022:03:06',
 '2022:03:07',
 '2022:03:08',
 '2022:03:09',
 '2022:03:10',
 '2022:03:11',
 '2022:03:12',
 '2022:03:13',
 '2022:03:14',
 '2022:03:15',
 '2022:03:16',
 '2022:03:17',
 '2022:03:25',
 '2022:03:27',
 '2022:03:28',
 '2022:03:29'}

In [10]:
start_date = datetime.strptime('2022:03:08','%Y:%m:%d')
end_date = datetime.strptime('2022:03:23','%Y:%m:%d')
type((end_date-start_date).days)
input_size

300

In [11]:

datas = {}
for city_string in city_set:
	city = data[city_string]
	start_date = datetime.strptime(city[0]['time'],'%Y:%m:%d')
	end_date = datetime.strptime(city[-1]['time'],'%Y:%m:%d')
	# start_date = datetime.strptime('2022:03:08','%Y:%m:%d')
	# end_date = datetime.strptime('2022:03:23','%Y:%m:%d')
	# CNT = []
	# VECTOR = []
	# while start_date < end_date:
	# 	start_date += timedelta(days=1)
	# 	cnt = 0
	# 	vector = []
	# 	for p in city:
	# 		if p['time']==(start_date).strftime('%Y:%m:%d'):
	# 			cnt+=1
	# 			vector.append(p['patient_vector'])

	# 	if any((type(v)==list for v in vector)):
	# 		vector = np.array([v for v in vector if type(v)==list])
	# 		vector = np.mean(vector,axis=0)
	# 	else:
	# 		vector = np.zeros((input_size,))
	# 	CNT.append(cnt)
	# 	VECTOR.append(vector)
	# CNT = np.array(CNT)
	# VECTOR = np.array(VECTOR)
	# temp = np.hstack((CNT.reshape(-1,1),VECTOR))
	period = (end_date-start_date).days+1
	CNT = np.zeros((period))
	VECTOR = np.zeros((period, input_size))
	cnt = 0
	vector = []
	day_idx = 0
	for p in city:
		if p['time'] != (start_date).strftime('%Y:%m:%d'):
			if any((type(v)==list for v in vector)):
				vector = np.array([v for v in vector if type(v)==list])
				VECTOR[day_idx] = np.mean(vector,axis=0)
			CNT[day_idx] = cnt
			cnt = 0
			vector = []
		while p['time'] != (start_date).strftime('%Y:%m:%d'):
			start_date += timedelta(days=1)
			day_idx += 1
		cnt += 1
		vector.append(p['patient_vector'])
	else:
		if any((type(v)==list for v in vector)):
			vector = np.array([v for v in vector if type(v)==list])
			VECTOR[day_idx] = np.mean(vector,axis=0)
		CNT[day_idx] = cnt
	temp = np.hstack((CNT.reshape(-1,1),VECTOR))
	datas[city_string] = torch.tensor(temp.reshape(-1,1,input_size+1)).float()
# input_size += 1

In [12]:
datas['重庆'].size()

torch.Size([253, 1, 301])

In [13]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
test_size = 1
len(datas)

11

In [15]:
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Net, self).__init__()
        self.gru = nn.GRU(input_size=input_size+1, \
            hidden_size=hidden_size)
        self.linear = nn.Linear(hidden_size,output_size) 
        self.relu = nn.ReLU()

    def forward(self, x):
        output, h_n = self.gru(x)
        # output = self.relu(output[-1])
        output = self.linear(output)
        output = self.relu(output[-1])
        return output

net = Net(input_size,16,1)
# net.to(device)

In [16]:

# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()


In [17]:
def data_iter(data,seq_len,test_size):
	indexs = np.arange(len(data)-seq_len-test_size)
	random.shuffle(indexs)
	for i in indexs:
		# print(data[i:i+7].shape)
		yield data[i:i+seq_len],data[i+seq_len][0][0]

In [18]:
epoches = 100
seq_len = 7
for epoch in tqdm(range(epoches)):  # loop over the dataset multiple times

    running_loss = 0.0
    for city in datas.values():
        for i,data in enumerate(data_iter(city,seq_len,test_size)):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # inputs, labels = inputs.to(device), labels.view(1,1).to(device)
            inputs, labels = inputs, labels.view(1,1)
            # raise Exception(inputs.size)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            # if labels != torch.tensor(0.,):
            #     raise Exception(labels.size(),labels)
            # else:
            #     raise Exception(inputs,labels)
            # if not i: print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

    if epoch%5 == 4:
        print(epoch+1,running_loss)

    running_loss = 0.0

print('Finished Training')

  5%|▌         | 5/100 [00:32<08:44,  5.52s/it]

5 7130.470533533259


 10%|█         | 10/100 [00:54<06:40,  4.45s/it]

10 5803.214492723833


 15%|█▌        | 15/100 [01:17<06:28,  4.57s/it]

...

90 5406.067693743345


 95%|█████████▌| 95/100 [09:11<00:30,  6.10s/it]

95 4386.063040872274


100%|██████████| 100/100 [09:41<00:00,  5.81s/it]

100 4040.710724876617
Finished Training


In [19]:
for i in range(test_size):
	print()
	for city_string in city_set:

		inputs = datas[city_string][-test_size-seq_len-1:-test_size-1]
		outputs = net(inputs)
		print(city_string,int(outputs)+1, \
			int(datas[city_string][-test_size-1][0][0]))


天水 1 3
南京 1 0
苏州 4 12
莆田 1 0
厦门 1 0
泉州 1 0
昆明 5 1
张掖 1 1
兰州 1 0
重庆 1 5
德宏傣族景颇族自治州瑞丽 1 0


In [20]:
# for city_string in city_set:

# 	inputs = datas[city_string][-9:-2]
# 	outputs = net(inputs)
# 	print(city_string,int(outputs)+1,int(datas[city_string][-2][0][0]))

In [21]:
# for i in net.gru.parameters():
# 	print(i)